## Problem Set 2, Solution
**Stats 507, Fall 2021**
*Dingyu Wang*
*September 28, 2021*

## Import

The remaining questions will use the following imports.

In [1]:
# modules: --------------------------------------------------------------------
import numpy as np
import pandas as pd
import re
# 79: -------------------------------------------------------------------------

## Question 3

In this question we will use Pandas to read, clean, and append several data files from the National Health and Nutrition Examination Survey NHANES.

### a) Read and append the demographic datasets

The target of the function is to 
* Choose specific columns and rename the columns with literate variable names
* Add an additional column identifying to which cohort each case belongs ("years" + "datasets name").
* Cope with missing data and convert each column to an appropriate type. 

In [4]:
def pd_demographic(name, year):
    '''
    Read and append the '.XPT' file of demographic datasets.

    This function will read the '.XPT' file and convert it to a DataFrame.
    Several columns are selected and renamed according to the meaning of the
    columns. Additional one column is added to the DataFrame and each column is
    convert into a appropriate type. Finally the function will return the
    processed DataFrame.

    Parameters
    ----------
    name: str
        The file's name.
    year:  str
        The conducted year of the file.

    Returns
    -------
    Processed DataFrame.
    '''
    df = pd.read_sas(name)
    columns = ['SEQN', 'RIDAGEYR', 'RIDRETH3', 'DMDEDUC2', 
        'DMDMARTL', 'RIDSTATR', 'SDMVPSU', 'SDMVSTRA', 'WTMEC2YR', 'WTINT2YR']
    columns_add = ['id', 'age', 'race', 'education',
                   'marital_status', 'interview_status', 
                   'pseudo_psu', 'pseudo_stratum', 
                   'exam_wt', 'interview_wt']
    df = df[columns]
    df = df.convert_dtypes()
    for i in range(3):
        df.iloc[:,i+3] = pd.Categorical(df.iloc[:,i+3])
    df.columns = columns_add
    cohort = 'NHANES' + ' ' + year
    df1 = pd.DataFrame({'cohort':[cohort for i in range(len(df.index))]}, 
                       index=df.index)
    df = pd.concat([df,df1], axis=1)             
    return df

Read the files directly from the website and apply the function abrove to each dataset and finally save the processed DataFrame into a pickle format.

In [5]:
url = 'https://wwwn.cdc.gov/Nchs/Nhanes/'
years = ['2011-2012', '2013-2014', '2015-2016', '2017-2018']
names_1 = ['DEMO_G.XPT', 'DEMO_H.XPT', 'DEMO_I.XPT', 'DEMO_J.XPT']
for i in range(4):
    name = url + years[i] +'/' + names_1[i]
    df = pd_demographic(name, years[i])
    file_name = years[i] + ' ' + 'demographic.pickle'
    df.to_pickle(file_name)  

### b) Read and append the  oral health and dentition data.

This part fairly do the same job as part a). One thing to mention is that I use regular expressionsin to find the columns with the format "OHXXXCTC" and "OHXXXTC".

In [6]:
def pd_health(name, year):
    '''
    Read and append the '.XPT' file of oral health and dentition datasets.

    This function will read the '.XPT' file and convert it to a DataFrame.
    Several columns are selected and renamed according to the meaning of the
    columns. Additional one column is added to the DataFrame and each column is
    convert into a appropriate type. Finally the function will return the
    processed DataFrame.

    Parameters
    ----------
    name: str
        The file's name.
    year:  str
        The conducted year of the file.

    Returns
    -------
    Processed DataFrame.
    '''
    df = pd.read_sas(name)
    columns_li = ['SEQN', 'OHDDESTS']
    column_1 = r'OHX\d\dCTC'
    column_2 = r'OHX\d\dTC'
    columns_li.extend(
        [m for m in df.columns if re.search(column_1,m) != None])
    columns_li.extend(
        [m for m in df.columns if re.search(column_2,m) != None])
    df = df[columns_li]
    columns_lower = [m.lower() for m in columns_li]
    columns_lower[0] = 'id'
    columns_lower[1] = 'dentition_code'
    df.columns = columns_lower
    df1 = df.convert_dtypes()
    for i in range(61):
        df1.iloc[:,i+1] = pd.Categorical(df1.iloc[:,i+1])
    cohort = 'NHANES' + ' ' + year
    df2 = pd.DataFrame({'cohort':[cohort for i in range(len(df.index))]}, 
                       index=df.index)
    df1 = pd.concat([df1,df2], axis=1)  
    return df1

Read the files directly from the website and apply the function abrove to each dataset and finally save the processed DataFrame into a pickle format.

In [7]:
url = 'https://wwwn.cdc.gov/Nchs/Nhanes/'
years = ['2011-2012', '2013-2014', '2015-2016', '2017-2018']
names_2 = ['OHXDEN_G.XPT', 'OHXDEN_H.XPT', 'OHXDEN_I.XPT', 'OHXDEN_J.XPT']
for i in range(4):
    name = url + years[i] +'/' + names_2[i]
    df = pd_health(name, years[i])
    file_name = years[i] + ' ' + 'oral health - dentition.pickle'
    df.to_pickle(file_name)

F:\python\lib\site-packages\pandas\io\sas\sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


### c) Number of cases there are in the two datasets

In this step I will read the data we saved in pickle format and then combine the dataset from different year together and reindex the DataFrame to calculate the number of cases in each datasets.

In [8]:
# calculate cases in each dataset: --------------------------------------------
df_demo = pd.concat([pd.read_pickle(years[i] + ' ' + 'demographic.pickle')
                     for i in range(4)])
df_demo = df_demo.reset_index()
df_demo = df_demo.drop(columns = ['index'])
df_ohxden = pd.concat(
    [pd.read_pickle(years[i] + ' ' + 'oral health - dentition.pickle')
     for i in range(4)
     ]
)
df_ohxden = df_ohxden.reset_index()
df_ohxden = df_ohxden.drop(columns = ['index'])

There are 39156 cases in the demographic datasets and 35909 cases in the ohxden datasets.

In [9]:
print(df_demo.shape)
print(df_ohxden.shape)

(39156, 11)
(35909, 63)


In the final step I will try to calculate the common cases shared by these two datasets according to 'unique id' column.

In [10]:
demo_id = np.array(df_demo['id'],dtype='int64')
ohxden_id = np.array(df_ohxden['id'],dtype='int64')
count_demo = np.bincount(demo_id)
count_ohxden = np.bincount(ohxden_id)
c = count_demo + count_ohxden
count = [idx for idx, val in enumerate(c) if val == 2]
len(count)

35909

There are 35909 cases are both in the demographic datasets and ohxden datasets.